#1. Library imports

In [1]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 6.5 MB 34.8 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 134 kB 31.9 MB/s 
     |████████████████████████████████| 212 kB 35.7 MB/s 
     |████████████████████████████████| 1.1 MB 31.4 MB/s 
     |████████████████████████████████| 127 kB 14.5 MB/s 
     |████████████████████████████████| 271 kB 26.8 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 30.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import tqdm
import nltk
import torch
import numpy as np
import pandas as pd
from torch import nn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import transformers

In [6]:
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import tqdm
from sklearn.metrics import confusion_matrix, classification_report

# 2. Data Imports

In [7]:
def load_data(split_name='train', columns=['text', 'stars'], folder='gdrive/MyDrive/COMP_4332/Project1/data'):
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [8]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

In [9]:
train_df = train_df[["text","stars"]]
valid_df = valid_df[["text","stars"]]

# MODEL BERT

## Tokenizer - REAL

### DataCollator

In [30]:
import datasets
import pandas as pd
import tensorflow as tf
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

In [32]:
def tokenize_function(x):
    return tokenizer(x, truncation=True)#return_tensors = 'pt'

def tokenize_df(df):
  tokens = df['text'].map(tokenize_function)
  df['input_ids'] = [x['input_ids'] for x in tokens]
  df['attention_mask'] = [x['attention_mask'] for x in tokens]
  df['token_type_ids'] = [x['token_type_ids'] for x in tokens]
  df.rename(columns={"stars":"labels"},inplace=True)
  return df

train_df = tokenize_df(train_df)
valid_df = tokenize_df(valid_df)
test_df = tokenize_df(test_df)

In [33]:
print(train_df.columns)
print(valid_df.columns)
print(test_df.columns)

Index(['text', 'labels', 'input_ids', 'attention_mask', 'token_type_ids'], dtype='object')
Index(['text', 'labels', 'input_ids', 'attention_mask', 'token_type_ids'], dtype='object')
Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'text', 'useful',
       'user_id', 'input_ids', 'attention_mask', 'token_type_ids'],
      dtype='object')


In [35]:
small_train = train_df.head(1000)
small_validate = valid_df.head(400)

In [36]:
import tempfile
import pathlib
import pyarrow as pa
import pyarrow.parquet as pq

In [37]:
table_train = pa.table({'labels': (list(small_train["labels"])), 'input_ids':list(small_train["input_ids"]), 'attention_mask':list(small_train["attention_mask"]),'token_type_ids':list(small_train["token_type_ids"]) })
table_validate = pa.table({'labels': list(small_validate["labels"]),'input_ids':list(small_validate["input_ids"]), 'attention_mask':list(small_validate["attention_mask"]),'token_type_ids':list(small_validate["token_type_ids"])})

In [38]:
#training = datasets.DatasetDict({"labels":list(train_df["labels"]), "text": list(train_df["text"])})
training = datasets.Dataset(table_train)
valid = datasets.Dataset(table_validate)

In [39]:
training.features
valid.features

{'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [40]:
MyDataset = datasets.DatasetDict({"train":training,"validation":valid})

In [41]:
MyDataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 400
    })
})

In [42]:
MyDataset = MyDataset.with_format("torch")

In [43]:
MyDataset["train"][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]),
 'input_ids': tensor([  101,   146,   112,  1396,  1151,  1303,   170,  8973,  1104,  1551,
          1208,  1105,   146,   112,  1396,  1309,  1151,  9333,   119,  1109,
          2094,  1110,  1579,  1363,  1105,  1103, 16096,  1132,  3613,   119,
          1573,  1677,  1139,  1160,  5095,  4454,  1132,  1103, 19278,  3202,
         15776, 24435,  1114,  1763,  4515,  1182,  1105,  1103

## Model - sequence

In [22]:
from transformers import AutoModelForSequenceClassification
 
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [23]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [24]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

In [25]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training,
    eval_dataset=valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


RuntimeError: ignored

In [ ]:
final_prediction = trainer.predict(validate)
print(final_prediction.shape, predictions.label_ids.shape)